In [1]:
from pycalphad import Database, Model
from pycalphad.core.utils import NumPyPrinter
from pycalphad.core.autograd_utils import build_functions as build_functions_ag
from pycalphad.core.algopy_utils import build_functions as build_functions_algo
from pycalphad.core.sympydiff_utils import make_gradient_from_graph
import pycalphad.variables as v
from sympy import lambdify
import numpy as np
import algopy
from algopy import UTPM
import itertools
mod = Model(Database('2016-04-01-AlNi.tdb'), ['AL', 'NI', 'VA'], 'FCC_L12')
print(tuple([v.T] + mod.site_fractions))

inp_arr = np.tile([[300,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,1], [600, 0.4, 0.6,0.4,0.6,0.4,0.6,0.4,0.6,1]], (300,1))
#inp_arr = np.array([[600, 0.4, 0.6,0.4,0.6,0.4,0.6,0.4,0.6,1]])
inp_arr = np.random.RandomState(1769).rand(10000, 10)
inp_arr[0, 0] = 1600
inp_arr[1, 0] = 1600
inp_arr[:, -1] = 1 - inp_arr[:, 1]
#inp_arr = np.expand_dims(inp_arr[1], axis=0)
print(inp_arr)

def grad_sympy(ast, variables):
    logical_np = [{'logical_and': np.logical_and}, np]
    return lambdify(tuple(variables), [ast.diff(v) for v in variables], dummify=True, modules=logical_np, printer=NumPyPrinter)

(T, FCC_L120AL, FCC_L120NI, FCC_L121AL, FCC_L121NI, FCC_L122AL, FCC_L122NI, FCC_L123AL, FCC_L123NI, FCC_L124VA)
[[  1.60000000e+03   3.70168537e-01   2.40990664e-01 ...,   7.33499088e-01
    8.72870784e-01   6.29831463e-01]
 [  1.60000000e+03   5.40478524e-01   5.16085552e-01 ...,   9.22020353e-01
    4.08862360e-01   4.59521476e-01]
 [  1.54755486e-01   8.87479862e-01   7.86040630e-01 ...,   7.90946657e-01
    5.03202313e-01   1.12520138e-01]
 ..., 
 [  1.27186493e-01   1.80890186e-01   2.34398113e-01 ...,   7.05099954e-01
    7.56157105e-02   8.19109814e-01]
 [  2.12752667e-01   5.47497839e-01   1.86760062e-01 ...,   1.78829310e-01
    3.31884816e-01   4.52502161e-01]
 [  2.19607372e-01   9.51496386e-01   7.94269100e-01 ...,   1.46878594e-01
    1.48560907e-02   4.85036135e-02]]


In [2]:
obj_ag, grad_ag, hess_ag = build_functions_ag(mod.ast, mod.variables)
obj_algo, grad_algo, hess_algo = build_functions_algo(mod.ast, mod.variables)
%time grad_numba = make_gradient_from_graph(mod)

CPU times: user 28.2 s, sys: 51 ms, total: 28.3 s
Wall time: 28.3 s


In [3]:
import itertools
%time g3 = grad_numba(*list(itertools.chain([inp_arr[..., i] for i in range(inp_arr.shape[-1])], [np.empty(inp_arr.shape[-1]), np.zeros(inp_arr.shape)])))

CPU times: user 23 ms, sys: 0 ns, total: 23 ms
Wall time: 23.3 ms


In [4]:
o1 = obj_ag(*inp_arr.T)
%time g1 = grad_ag(*inp_arr.T)
%time h1 = hess_ag(*inp_arr.T)
o2 = obj_algo(*inp_arr.T)
%time g2 = grad_algo(*inp_arr.T)
h2 = hess_algo(*inp_arr.T)

CPU times: user 240 ms, sys: 53 ms, total: 293 ms
Wall time: 293 ms
CPU times: user 3.84 s, sys: 158 ms, total: 3.99 s
Wall time: 3.99 s
CPU times: user 1.57 s, sys: 7 ms, total: 1.57 s
Wall time: 1.57 s


In [5]:
g1

array([[ -3.04586555e+01,  -6.86365608e+04,  -7.75780824e+04, ...,
         -2.17832811e+04,  -2.81337508e+04,  -1.00238564e+05],
       [ -3.20865747e+01,  -4.67034156e+04,  -5.65974961e+04, ...,
         -3.79620528e+04,  -4.48707719e+04,  -1.42145241e+05],
       [ -2.17009482e+00,   1.50083732e-01,   1.29397685e-01, ...,
          1.30458093e-01,   5.33842274e-02,  -1.77927238e-01],
       ..., 
       [ -4.24260733e+00,  -1.58921800e-01,  -1.00908606e-01, ...,
          1.45650238e-01,  -3.54192321e-01,  -4.56945407e-01],
       [ -2.99337125e+00,   1.13624247e-01,  -1.93734217e-01, ...,
         -2.06134742e-01,  -2.94252716e-02,  -4.11533833e-01],
       [ -1.45762141e+00,   2.22283475e-01,   1.80035521e-01, ...,
         -2.14767440e-01,  -7.50709241e-01,  -1.64030233e-01]])

In [6]:
g2

array([[ -3.36190308e+00,   1.50568090e+01,  -1.02673771e+03, ...,
          1.67500444e+03,   2.09725912e+03,  -3.92582721e+03],
       [ -3.18344881e+00,   8.30543231e+02,   7.30838325e+02, ...,
          1.98365960e+03,   2.28034303e+02,  -3.30645187e+03],
       [ -2.17009482e+00,   1.50083732e-01,   1.29397685e-01, ...,
          1.30458093e-01,   5.33842274e-02,  -1.77927238e-01],
       ..., 
       [ -4.24260733e+00,  -1.58921800e-01,  -1.00908606e-01, ...,
          1.45650238e-01,  -3.54192321e-01,  -4.56945407e-01],
       [ -2.99337125e+00,   1.13624247e-01,  -1.93734217e-01, ...,
         -2.06134742e-01,  -2.94252716e-02,  -4.11533833e-01],
       [ -1.45762141e+00,   2.22283475e-01,   1.80035521e-01, ...,
         -2.14767440e-01,  -7.50709241e-01,  -1.64030233e-01]])

In [7]:
np.array(g3)

array([[ -3.04586555e+01,  -6.86365608e+04,  -7.75780824e+04, ...,
         -2.17832811e+04,  -2.81337508e+04,  -1.00238564e+05],
       [ -3.20865747e+01,  -4.67034156e+04,  -5.65974961e+04, ...,
         -3.79620528e+04,  -4.48707719e+04,  -1.42145241e+05],
       [ -2.17009482e+00,   1.50083732e-01,   1.29397685e-01, ...,
          1.30458093e-01,   5.33842274e-02,  -1.77927238e-01],
       ..., 
       [ -4.24260733e+00,  -1.58921800e-01,  -1.00908606e-01, ...,
          1.45650238e-01,  -3.54192321e-01,  -4.56945407e-01],
       [ -2.99337125e+00,   1.13624247e-01,  -1.93734217e-01, ...,
         -2.06134742e-01,  -2.94252716e-02,  -4.11533833e-01],
       [ -1.45762141e+00,   2.22283475e-01,   1.80035521e-01, ...,
         -2.14767440e-01,  -7.50709241e-01,  -1.64030233e-01]])

In [8]:
import numpy.testing

numpy.testing.assert_allclose(o1, o2)
numpy.testing.assert_allclose(g1, g2)
numpy.testing.assert_allclose(h1, h2)
print('equivalent')

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

(mismatch 0.01999999999999602%)
 x: array([[ -3.045866e+01,  -6.863656e+04,  -7.757808e+04, ...,
         -2.178328e+04,  -2.813375e+04,  -1.002386e+05],
       [ -3.208657e+01,  -4.670342e+04,  -5.659750e+04, ...,...
 y: array([[ -3.361903e+00,   1.505681e+01,  -1.026738e+03, ...,
          1.675004e+03,   2.097259e+03,  -3.925827e+03],
       [ -3.183449e+00,   8.305432e+02,   7.308383e+02, ...,...